In [8]:
import csv
import numpy as np
import cv2
from time import time
import os
from PIL import Image as pilimg

In [9]:
person_labels=['/m/04yx4', '/m/03bt1vf', '/m/01bl7v', '/m/05r655', '/m/01g317'] #merger for 'person'
bag_labels=['/m/0hf58v5', '/m/01940j'] #merger for 'bag'

In [10]:
root_dir='/mnt/openimgv4/'
lbl_dir=root_dir+'tools/validation-annotations-bbox_people_and_bag.csv'
cls_dir_num=root_dir+'clsDict_new.csv'
cls_dir_code=root_dir+'train/label/challenge-2018-class-descriptions-500.csv'
dat_dir=root_dir+'sources/valid_img/'
rec_dir=root_dir+'sources/rec_files/'

In [11]:
# convert csv file to dict
def row_csv2dict(csv_file):
    dict_club={}
    with open(csv_file)as f:
        reader=csv.reader(f,delimiter=',')
        for row in reader:
            dict_club[row[0]]=row[1]
    return dict_club

In [12]:
# search for class name for a given label
def get_cls(lbl):
    cls_dict=row_csv2dict(cls_dir_code)
    if lbl in cls_dict:
        return cls_dict[lbl]
    else:
        return 'None' # return 'None' for empty labels

In [13]:
# rotate vertical images
def rotatecc(id):
    im = pilimg.open(dat_dir+id+'.jpg')
    ng = im.transpose(pilimg.ROTATE_90)
    ng.save(dat_dir+id+'_r.jpg')
    with open(root_dir+'sources/valid_rotated_cc.log', 'a+') as f:
        f.write(id+'\n')

In [14]:
# search for labels for a given image id
id_idx=0 #index of image id
lbl_idx=[2, 4, 5, 6, 7] #label_name, xmin, xmax, ymin, ymax

# only preserve non-group labels
gp_idx=10

start=time()
with open(lbl_dir,encoding='utf-8') as cfile:
    reader = csv.reader(cfile)
    readeritem=[]
    readeritem.extend([row for row in reader])
with open(rec_dir+'valid_person_and_bag.lst', 'w+') as f:
    isgot=0
    id_old='666'
    img_idx=0
    for i,rows in enumerate(readeritem):
        if i%1000==0:
            print(i,' time:',time()-start)
        path = dat_dir + rows[id_idx] + '.jpg'
        if os.path.isfile(path):
            if True:
                if rows[id_idx]!=id_old:
                    if id_old!='666':
                        if box_count>0:
                            f.write(
                            str(img_idx) + '\t' +
                            str(4) + '\t' + str(5) + '\t' +
                            str(W) + '\t' + str(H) + '\t' +
                            mcstr +
                            str(file_id) + '.jpg\n'
                            )
                            img_idx+=1
                    mcstr=''
                    box_count=0
                    img=cv2.imread(path)
                    W=np.size(img,1)
                    H=np.size(img,0)
                    row=rows
                    id_old=rows[id_idx]
                    if H>W:
                        file_id=rows[id_idx]+'_r'
                        rotatecc(rows[id_idx])
                        tmp=W
                        W=H
                        H=tmp
                        isRotated=True
                    else:
                        file_id=rows[id_idx]
                        isRotated=False
                        
                if rows[2] in person_labels:
                    mcstr=mcstr+str(0)+'\t' #target_labels.index(int(lbls[j,0]))
                    if isRotated==False:
                        mcstr+=str(rows[4]) + '\t' + str(rows[6]) + '\t' 
                        mcstr+=str(rows[5]) + '\t' + str(rows[7]) + '\t'
                    else:
                        mcstr+=str(rows[6]) + '\t' + str(1-float(rows[5])) + '\t' 
                        mcstr+=str(rows[7]) + '\t' + str(1-float(rows[4])) + '\t'
                    box_count+=1
                    
                if rows[2] in bag_labels:
                    mcstr=mcstr+str(1)+'\t' #target_labels.index(int(lbls[j,0]))
                    if isRotated==False:
                        mcstr+=str(rows[4]) + '\t' + str(rows[6]) + '\t' 
                        mcstr+=str(rows[5]) + '\t' + str(rows[7]) + '\t'
                    else:
                        mcstr+=str(rows[6]) + '\t' + str(1-float(rows[5])) + '\t' 
                        mcstr+=str(rows[7]) + '\t' + str(1-float(rows[4])) + '\t'
                    box_count+=1


0  time: 0.05183076858520508
1000  time: 10.342251062393188
2000  time: 21.174380779266357
3000  time: 32.42556381225586
4000  time: 42.688151359558105
5000  time: 51.77247190475464
6000  time: 61.618523597717285
7000  time: 72.29511618614197
8000  time: 82.960120677948
9000  time: 91.73302745819092
10000  time: 102.00993514060974
11000  time: 110.31768131256104
12000  time: 120.97136449813843
13000  time: 129.4767246246338
14000  time: 137.76612782478333
15000  time: 146.6457769870758
16000  time: 159.41065382957458
17000  time: 167.42717576026917
18000  time: 175.87838172912598
19000  time: 184.5706160068512
20000  time: 193.46529817581177
21000  time: 202.60886406898499
22000  time: 211.27645015716553


In [46]:
#def convert2array(csv_dat):
    # out=np.reshape(['0','0','0','0','0','0','0','0','0','0','0',],(1,11))
    # for i,rows in enumerate(csv_dat):
        # if i!=0: 
            # #print(i)
            # tmp=['0']
            # row=rows
            # for k in range(11):
                # tmp.append(str(row[k]))
            # tmp=tmp[1:]
            # out=np.append(out,np.reshape(tmp,(1,11)),axis=0)
    # out=np.delete(out,0,axis=0)
    # return out # every 5 values (1 row) corresponds to 1 bbox

## loop through all items in dat_dir (jpg)
img_idx = 0